In [1]:
# setting the website that should be scraped 
website = 'https://www.morizon.pl/mieszkania/krakow/'

In [2]:
# downloading Selenium libraries 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# downloading BeautifulSoup library 
from bs4 import BeautifulSoup

# downloading Numpy & Pandas libraries
import pandas as pd
import numpy as np

# downloading additional libraries
from requests import get
import requests, openpyxl
import re 

In [15]:
# setting Selenium options and opening browser in the background 
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [4]:
links = []

# opening start website in the browser and saving the HTML code
driver.get(website)
html=driver.page_source

# repacing tricky advertisement link with n/a so it will not be scraped 
html = html.replace("https://www.lendi.pl/formularz/kredyty-hipoteczne?hypothecValue=539900.00&loanPurpose.what=Apartment&loanPurpose.market=SecondaryMarket&meeting_voivoidship=małopolskie&meeting_city=Kraków&utm_entry_page=https://www.mori","Unspecified")
soup=BeautifulSoup(html,'html.parser')

# scraping div with subcategories 
subcategories = soup.find("div", {"id": "locationListChildren"})

# scraping list containg links to subcategories 
for link in subcategories.find_all("a", href=re.compile("/mieszkania/")):
    links.append(link.get('href'))
    
# deleting duplicated links
links = list(dict.fromkeys(links))

# creating full links 
i = "https://www.morizon.pl"
full_links = []

for link in links:
    url = (str(i) + link)
    full_links.append(url)

In [5]:
full_links

['https://www.morizon.pl/mieszkania/krakow/podgorze/',
 'https://www.morizon.pl/mieszkania/krakow/krowodrza/',
 'https://www.morizon.pl/mieszkania/krakow/stare-miasto/',
 'https://www.morizon.pl/mieszkania/krakow/pradnik-bialy/',
 'https://www.morizon.pl/mieszkania/krakow/biezanow-prokocim/',
 'https://www.morizon.pl/mieszkania/krakow/grzegorzki/',
 'https://www.morizon.pl/mieszkania/krakow/czyzyny/',
 'https://www.morizon.pl/mieszkania/krakow/debniki/',
 'https://www.morizon.pl/mieszkania/krakow/podgorze-duchackie/',
 'https://www.morizon.pl/mieszkania/krakow/nowa-huta/',
 'https://www.morizon.pl/mieszkania/krakow/mistrzejowice/',
 'https://www.morizon.pl/mieszkania/krakow/pradnik-czerwony/',
 'https://www.morizon.pl/mieszkania/krakow/zwierzyniec/',
 'https://www.morizon.pl/mieszkania/krakow/bronowice/',
 'https://www.morizon.pl/mieszkania/krakow/bienczyce/',
 'https://www.morizon.pl/mieszkania/krakow/lagiewniki/',
 'https://www.morizon.pl/mieszkania/krakow/wzgorza-krzeslawickie/',
 '

In [6]:
# Optional - getting subcategories of subcategories 

for link in full_links:
    # opening start website in the browser and saving the HTML code
    driver.get(link)
    html=driver.page_source
    
    # repacing tricky advertisement link with n/a so it will not be scraped 
    html = html.replace("https://www.lendi.pl/formularz/kredyty-hipoteczne?hypothecValue=539900.00&loanPurpose.what=Apartment&loanPurpose.market=SecondaryMarket&meeting_voivoidship=małopolskie&meeting_city=Kraków&utm_entry_page=https://www.mori","Unspecified")
    soup=BeautifulSoup(html,'html.parser')
    
    # scraping div with subcategories 
    subcategories = soup.find("div", {"id": "locationListChildren"})
    
    # scraping list containg links to subcategories 
    links_sub = []
    for link in subcategories.find_all("a", href=re.compile("/mieszkania/")):
        links_sub.append(link.get('href'))
        
    # deleting duplicated links
    links_sub = list(dict.fromkeys(links_sub))
    i = "https://www.morizon.pl"
    full_links_updated = []
    
    # creating full links 
    for link in links_sub:
        url = (str(i) + link)
        full_links_updated.append(url)

In [7]:
def substring_after(s, delim):
   return s.partition(delim)[2]

In [8]:
pages = []
pages2 = []

for link2 in full_links:
    driver.get(link2)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    
    flats=[]
    
    if soup.find("ul", {"class": "nav nav-pills mz-pagination-number"}):
        test = soup.find("ul", {"class": "nav nav-pills mz-pagination-number"})

        for link in test.find_all("a", href=re.compile("/mieszkania/")):
            flat = link.get('href')
            flat = substring_after(flat, "page=") 
            flats.append(flat)
            flats2 = list(map(int, flats))

        maxval = max(flats2)
        # print(maxval)

        i = 0
        for j in range(0, maxval):
            i = i + 1
            url = (str(link2) + str("?page=") + str(i))
            # print(url)
            pages2.append(url)
            pages = pages2
    else:
        maxval = 1
        
        i = 0
        for j in range(0, maxval):
            i = i + 1
            url = (str(link2) + str("?page=") + str(i))
            # print(url)
            pages2.append(url)
            pages = pages2

In [11]:
pages

['https://www.morizon.pl/mieszkania/krakow/podgorze/?page=1',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=2',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=3',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=4',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=5',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=6',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=7',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=8',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=9',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=10',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=11',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=12',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=13',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=14',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=15',
 'https://www.morizon.pl/mieszkania/krakow/podgorze/?page=16',
 

In [16]:
flats = []

for page in pages:
    driver.get(page)
    html=driver.page_source
        
    # repacing advertisement link with n/a so it will not be scraped 
    html = html.replace("https://www.lendi.pl/formularz/kredyty-hipoteczne?hypothecValue=539900.00&loanPurpose.what=Apartment&loanPurpose.market=SecondaryMarket&meeting_voivoidship=małopolskie&meeting_city=Kraków&utm_entry_page=https://www.mori","Unspecified")
    soup=BeautifulSoup(html,'html.parser')
    
    for link in soup.find_all("a", href=re.compile("www.morizon.pl/oferta/sprzedaz-mieszkanie")):
        flats.append(link.get('href'))    
            
    # deleting duplicated links
flats = list(dict.fromkeys(flats)) # deleting duplicates

In [17]:
#Creating excel file to save the results
excel = openpyxl.Workbook()
sheet = excel.active

#Changing sheet name
sheet.title = 'Morizonflats'

#Adding rows that will be columns' names
sheet.append(['Total_Price', 'Price_per_Square_Meter', 'Total_living_area', 'Number_of_Rooms', 'Offer_Link', 'Latitude', 'Longitude', 'Developer_name', 'Walls_height', 'Investment_name', 'Floor', 'Total_number_of_floors', 'Kitchen_type', 'Number_of_bathrooms', 'Toilet_and_WC_type', 'Balcony', 'Total_area_of_Balcony', 'Total_area_of_Garden', 'Market_type', 'Available_from','Type_of_property', 'Type_of_Contract', 'Offer_ID', 'Date_of_last_update', 'Date_of_publication', 'Number_of_levels', 'Bedroom_area', 'Bathroom_area', 'Description'])

In [18]:
for link in flats:
    #opening the link to the flat
    driver.get(link)
    html=driver.page_source
        
    #cleaning the html file for easier scraping
    html = html.replace("Cena"," ")
    html = html.replace("za m²"," ")    
    html = html.replace("Powierzchnia"," ")    
    html = html.replace("Pokoje"," ")    
    soup=BeautifulSoup(html,'html.parser')
        
    # scraping price
    if soup.find('li', attrs={'class':'paramIconPrice'}):
        price = soup.find('li', attrs={'class':'paramIconPrice'}).text.strip()
        # cleaning price variable so it will only cointan numbers 
        price = price.replace("zł"," ")
    else: price = "N/A"
        
    # scraping price
    if soup.find('li', attrs={'class':'paramIconPriceM2'}):
        pricem2 = soup.find('li', attrs={'class':'paramIconPriceM2'}).text.strip()
        # cleaning price variable so it will only cointan numbers 
        pricem2 = pricem2.replace("zł"," ")
    else: pricem2 = "N/A"
        
    # scraping price per sqm2
    if soup.find('li', attrs={'class':'paramIconLivingArea'}):
        livingarea = soup.find('li', attrs={'class':'paramIconLivingArea'}).text.strip()
        # cleaning price variable so it will only cointan numbers 
        livingarea = livingarea.replace("m²"," ")
    else: livingarea = "N/A"
        
    # scraping price
    if soup.find('li', attrs={'class':'paramIconNumberOfRooms'}):
        rooms = soup.find('li', attrs={'class':'paramIconNumberOfRooms'}).text.strip()
    else: rooms = "N/A"
        
    # scraping price
    if soup.find('li', attrs={'class':'paramIconPrice'}):
        price = soup.find('li', attrs={'class':'paramIconPrice'}).text.strip()
    else: price = "N/A"
        
    # scraping price
    if soup.find('li', attrs={'class':'paramIconPrice'}):
        price = soup.find('li', attrs={'class':'paramIconPrice'}).text.strip()
    else: price = "N/A"
        
    # scraping price
    if soup.find('li', attrs={'class':'paramIconPrice'}):
        price = soup.find('li', attrs={'class':'paramIconPrice'}).text.strip()
    else: price = "N/A"
        
    # scraping price
    if soup.find('li', attrs={'class':'paramIconPrice'}):
        price = soup.find('li', attrs={'class':'paramIconPrice'}).text.strip()
    else: price = "N/A"
        
    #########################################################################################################################################################################################
        
    link2 = link
        
    # cleaning price variable so it will only cointan numbers 
    #pricem2 = pricem2.replace("zł"," ")
        
    # cleaning price variable so it will only cointan numbers 
    # livingarea = livingarea.replace("m²"," ")
        
    # scraping description 
    if soup.find("div", {"class": "description"}).text.strip():
        opis = soup.find("div", {"class": "description"}).text.strip()
    else:
        opis = "N/A"
    
    try:
        lat = soup.find("div", class_="GoogleMap")["data-lat"]
    except:
        lat = "N/A"
        
    try:
        lng = soup.find("div", class_="GoogleMap")["data-lng"]
    except:
        lng = "N/A"
        
    ##scraping latitute and longtitude 
    #if soup.find("div", class_="GoogleMap")["data-lat"]:
    #    lat = soup.find("div", class_="GoogleMap")["data-lat"]
    #    lng = soup.find("div", class_="GoogleMap")["data-lng"]
    #else:
    #    lat = "N/A"
    #   lng = "N/A"
        
    ##########################################################################
        
    # scraping a table with additional informations  
    div=soup.find_all('table')
    
    table0 = pd.DataFrame()
    
    try:
        
        table0=pd.read_html(str(div[0]))

        # transforming the table to string
        ans = np.array(table0, dtype="object")
        ans = (ans[0])

        # saving array as a dataframe 
        df = pd.DataFrame(ans)

        df = df.rename(columns={0: "a", 1: "b"})

        # pivoting table 
        table0 = pd.pivot_table(df, values='b', columns=['a'], aggfunc=np.sum)

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Deweloper:" in table0.columns:
            Deweloper = table0['Deweloper:'].iloc[0]
        else: Deweloper = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Wysokość wnętrza:" in table0.columns:
            Wysokosc = table0['Wysokość wnętrza:'].iloc[0]
        else: Wysokosc = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Inwestycja:" in table0.columns:
            Inwestycja = table0['Inwestycja:'].iloc[0]
        else: Inwestycja = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Powierzchnia użytkowa:" in table0.columns:
            Powierzchnia = table0['Powierzchnia użytkowa:'].iloc[0]
        else: Powierzchnia = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Piętro:" in table0.columns:
            Pietro = table0['Piętro:'].iloc[0]
        else: Pietro = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Liczba pięter:" in table0.columns:
            Lpieter = table0['Liczba pięter:'].iloc[0]
        else: Lpieter = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Typ kuchni:" in table0.columns:
            Kuchnia = table0['Typ kuchni:'].iloc[0]
        else: Kuchnia = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Liczba łazienek:" in table0.columns:
            Lazienki = table0['Liczba łazienek:'].iloc[0]
        else: Lazienki = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Czy łazienka z WC:" in table0.columns:
            LazWC = table0['Czy łazienka z WC:'].iloc[0]
        else: LazWC = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Balkon:" in table0.columns:
            Balkon = table0['Balkon:'].iloc[0]
        else: Balkon = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Powierzchnia balkonu:" in table0.columns:
            Powbalkonu = table0['Powierzchnia balkonu:'].iloc[0]
        else: Powbalkonu = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Powierzchnia ogródka:" in table0.columns:
            PowOgrodka = table0['Powierzchnia ogródka:'].iloc[0]
        else: PowOgrodka = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Rynek:" in table0.columns:
            Rynek = table0['Rynek:'].iloc[0]
        else: Rynek = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Dostępne od:" in table0.columns:
            Dostod = table0['Dostępne od:'].iloc[0]
        else: Dostod = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Forma własności:" in table0.columns:
            Formawlasnosci = table0['Forma własności:'].iloc[0]
        else: Formawlasnosci = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Rodzaj umowy:" in table0.columns:
            RodzajUmowy = table0['Rodzaj umowy:'].iloc[0]
        else: RodzajUmowy = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Numer oferty:" in table0.columns:
            NumOferty = table0['Numer oferty:'].iloc[0]
        else: NumOferty = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Zaktualizowano:" in table0.columns:
            Zaktualizowano = table0['Zaktualizowano:'].iloc[0]
        else: Zaktualizowano = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Opublikowano:" in table0.columns:
            Opublikowano = table0['Opublikowano:'].iloc[0]
        else: Opublikowano = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Liczba poziomów mieszkania:" in table0.columns:
            PozMiesz = table0['Liczba poziomów mieszkania:'].iloc[0]
        else: PozMiesz = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Liczba sypialni:" in table0.columns:
            Sypialn = table0['Liczba sypialni:'].iloc[0]
        else: Sypialn = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Na biuro:" in table0.columns:
            Biur = table0['Na biuro:'].iloc[0]
        else: Biur = "N/A"

        # scraping information about developer (if applicable - else filling it with 'N/A')
        if "Powierzchnia łazienki:" in table0.columns:
            Lazienka = table0['Powierzchnia łazienki:'].iloc[0]
        else: Lazienka = "N/A"
        

        ###############################################################################

        # appending all scraped variables to an excel file 
        sheet.append([price, pricem2, livingarea, rooms, link2, lat, lng, Deweloper, Wysokosc, Inwestycja, Pietro, Lpieter, Kuchnia, Lazienki, LazWC, Balkon, Powbalkonu, PowOgrodka, Rynek, Dostod, Formawlasnosci, RodzajUmowy, NumOferty, Zaktualizowano, Opublikowano, PozMiesz, Sypialn, Lazienka, opis])
    except:
        print('n/a')
        
        
# saving excel file 
excel.save('Morizon_flats.xlsx')

n/a
n/a


KeyboardInterrupt: 

In [19]:
excel.save('Morizon_flats.xlsx')